## Recurrent Neural Network Model

In [10]:
'''
Step 1: Loading MNIST Train Dataset
'''

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# download train dataset

train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform = transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor()) # download not required, already downloaded in last step


'''
Step 2: Make Dataset Iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs) 
num_epochs

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)


'''
Step 3: Create Model Class
'''

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        
        # Hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        #Building RNN
        # batch_first = True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first = True, nonlinearity='relu')
        
        # readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
        
        
    def forward(self,x):
        # Initialize hidden state with zeroes
        # (layer_dim, batch_size, hidden_dim)
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        
        # One time step
        out, hn = self.rnn(x,h0)
        
        # Index hiddent state of last time step
        # out.size() --> 100,28,100
        # out[:,-1,:] --> 100, 100 --> just want last time step hidden states
        out = self.fc(out[:, -1, :])
        
        # out.size() --> 100, 10

        return out

    
'''
Step 4: Instantiate Model Class
'''

input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


'''
Step 7: Train Model
'''

# Number of steps to unroll
seq_dim = 28

iter = 0

for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        # load images as varible
        images = Variable(images.view(-1, seq_dim, input_dim))
        labels = Variable(labels)
        
        # clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output
        outputs = model(images)
        
        # Calculate Loss : softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # getting gradients w.r.t parameters
        loss.backward()
        
        # updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # calculate Accuracy
            correct = 0
            total = 0
            #iterate through test dataset
            for images, labels in test_loader:
                #load images to a Torch variable
                images = Variable(images.view(-1, seq_dim, input_dim))
                
                # Forward pass only to get outputs
                outputs = model(images)
                
                # get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels 
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = float(100 * correct / total)
            
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))



Iteration: 500. Loss: 1.610021948814392. Accuracy: 39.0
Iteration: 1000. Loss: 1.3803597688674927. Accuracy: 36.0
Iteration: 1500. Loss: 0.7966611385345459. Accuracy: 73.0
Iteration: 2000. Loss: 0.5110621452331543. Accuracy: 83.0
Iteration: 2500. Loss: 0.6753219366073608. Accuracy: 84.0
Iteration: 3000. Loss: 0.6947190761566162. Accuracy: 77.0
